# BI272

Updating the dataset: P117290200000

* LiF 1A is the guide channel and its count rate is rock steady.  
* The other channels are brighter, but their count rates vary a bit.  
* We assume that they are contaminated by a nearby star.  
* LiF 1B suffers from the worm. It cannot be corrected, so we omit it.
* We scale all four channels to match LiF 2B, then use LiF 2A to determine the scale factor between FUSE and STIS.

COMMENT File updated 27 September 2022<br>
COMMENT This NVO file has been reconstructed from the corresponding ALL file.<br>
COMMENT All segments have been scaled to match STIS.<br>
COMMENT The region between 1180 and 1190, affected by the worm, is omitted.

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E13

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset in pixels between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(7))
    error_spec = convolve(error_spec, Box1DKernel(7))
    flux_ref   = convolve(flux_ref,   Box1DKernel(7))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(20)
    for i in range(20):
        j = i - 10
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 10 - np.argmin(chisq)

In [ ]:
# Read header keywords.

filename = 'p117290200000all4ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# Guider channel is LiF 1.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

from specutils.manipulation import (box_smooth)

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed spectra.  
# Note that spectra are scaled to have values near unity.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A')
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()
ax.set_ylim([0,25])

# LiF 1A is the guide channel, but all the other spectra are brighter.
# The LiF 1A count rate is rock steady, while the others vary a bit,
# suggesting that they are contaminated by a second star.

In [ ]:
# Normally, we would scale the other spectra to match LiF 1A, but we will see that 
# LiF 2A is a better choice when scaling the FUSE spectrum to match STIS.
# For now, let us scale the other three channels to match LiF 2B.

# Select a broad spectral region.

region = SpectralRegion(1042*u.AA, 1062*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_lif2b = extract_region(lif2b, region)
sub_sic1a = extract_region(sic1a, region)
sub_sic2b = extract_region(sic2b, region)

# Compute ratio of their fluxes to LiF 2B.

mean = sub_lif2b.mean()
scale_lif1 = mean/sub_lif1a.mean()
print ('Scale LiF 1A by', scale_lif1)

scale_sic1 = mean/sub_sic1a.mean()
print ('Scale SiC 1A by', scale_sic1)

scale_sic2 = mean/sub_sic2b.mean()
print ('Scale SiC 2B by', scale_sic1)

In [ ]:
# Scale the smoothed spectra.

lif1a_bsmooth *= scale_lif1
sic1a_bsmooth *= scale_sic1
sic2b_bsmooth *= scale_sic2

In [ ]:
# Plot the rescaled spectra.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()
ax.set_ylim([0,25])
ax.set_xlim([1000,1060])

# Because all of the extracted segments have the same exposure time, we can use these values
# for the long- and short-wavelength regions as well.

In [ ]:
# Compare with a STIS spectrum of the same star.

filename = 'oe9s1s030_x1d.fits'
print ('Target:', fits.getval(filename, 'TARGNAME'))
print ('Aperture:', fits.getval(filename, 'APERTURE'))
stis = Spectrum1D.read(filename, format="HST/STIS")
stis_bsmooth = box_smooth(stis, width=15)

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.plot(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.set_xlim([1120,1200])
ax.set_ylim([0,15])
ax.legend()

# The LiF 2A spectrum looks pretty good.
# At the longest wavelengths, the worm imposes a slope on the LiF 1B spectrum that we cannot correct
# simply by rescaling it.  We will omit this segment from the NVO file.

In [ ]:
# How much brigher is LiF 2A than STIS?

# Select a spectral region.

region = SpectralRegion(1120*u.AA, 1145*u.AA)
sub_lif2a = extract_region(lif2a, region)
sub_lif1b = extract_region(lif1b, region)

# Compute ratio of their fluxes.

scale_entire_spectrum = sub_lif1b.mean()/sub_lif2a.mean()
print ('Scale LiF 2A by', scale_entire_spectrum)

# We will apply this factor to the entire spectrum.

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 1A.

shift_lif2 = compute_shift(lif2b_data, lif1a_data, 1040, 1070)
shift_sic1 = compute_shift(sic1a_data, lif1a_data, 1040, 1070)
shift_sic2 = compute_shift(sic2b_data, lif1a_data, 1040, 1070)

print ('Shift LiF 2 by ', shift_lif2, ' pixels.')
print ('Shift SiC 1 by ', shift_sic1, ' pixels.')
print ('Shift SiC 2 by ', shift_sic2, ' pixels.')

In [ ]:
# Shift all eight spectra.

shift_lif2 *= WPC
shift_sic1 *= WPC
shift_sic2 *= WPC

lif1a_shift = lif1a
lif1b_shift = lif1b
lif2a_shift = Spectrum1D(spectral_axis=lif2a.spectral_axis + shift_lif2 * u.AA, flux=lif2a.flux)
lif2b_shift = Spectrum1D(spectral_axis=lif2b.spectral_axis + shift_lif2 * u.AA, flux=lif2b.flux)
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)
sic2a_shift = Spectrum1D(spectral_axis=sic2a.spectral_axis + shift_sic2 * u.AA, flux=sic2a.flux)
sic2b_shift = Spectrum1D(spectral_axis=sic2b.spectral_axis + shift_sic2 * u.AA, flux=sic2b.flux)

f, ax = plt.subplots()  
ax.step(lif1a.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_shift.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_shift.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1060,1070])
ax.set_ylim([0,20])

In [ ]:
# Reassemble the NVO file.  Set the SiC channels and the 1180-1190 A region to zero.

filename = 'p117290200000nvo4ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

data['flux'] = 0.
data['error'] = 0.

# Scale SiC 1B
sic1b_wave = sic1b_shift.spectral_axis.value
h = np.where((sic1b_wave > 899.99) & (sic1b_wave < 917.5))
g = np.where((data['wave'] > np.min(sic1b_wave[h]) - 0.01) & (data['wave'] < 917.5))
data['flux'][g] = float(scale_sic1) * sic1b_data['flux'][h]
data['error'][g] = float(scale_sic1) * sic1b_data['error'][h]

# Scale SiC 2A
sic2a_wave = sic2a_shift.spectral_axis.value
g = np.where((data['wave'] > 917.5) & (data['wave'] < 998))
h = np.where((sic2a_wave > 917.5) & (sic2a_wave < 998))
data['flux'][g] = float(scale_sic2) * sic2a_data['flux'][h]
data['error'][g] = float(scale_sic2) * sic2a_data['error'][h]

# Scale LiF 1A
lif1a_wave = lif1a_shift.spectral_axis.value
g = np.where((data['wave'] > 998) & (data['wave'] < 1082.5))
h = np.where((lif1a_wave > 998) & (lif1a_wave < 1082.5))
data['flux'][g] = float(scale_lif1) * lif1a_data['flux'][h]
data['error'][g] = float(scale_lif1) * lif1a_data['error'][h]

# Scale SiC 1A 
sic1a_wave = sic1a_shift.spectral_axis.value
g = np.where((data['wave'] > 1082.5) & (data['wave'] < 1087.5))
h = np.where((sic1a_wave > 1082.5) & (sic1a_wave < 1087.5))
data['flux'][g] = float(scale_sic1) * sic1a_data['flux'][h]
data['error'][g] = float(scale_sic1) * sic1a_data['error'][h]

# Scale LiF 2A
lif2a_wave = lif2a_shift.spectral_axis.value
g = np.where((data['wave'] > 1087.5) & (data['wave'] < 1180))
h = np.where((lif2a_wave > 1087.5) & (lif2a_wave < 1180))
data['flux'][g] = lif2a_data['flux'][h]
data['error'][g] = lif2a_data['error'][h]

# Rescale entire spectrum to match STIS.
data['flux'] *= float(scale_entire_spectrum)
data['error'] *= float(scale_entire_spectrum)

hdr['comment'] = ''
hdr['comment'] = 'File updated 27 September 2022'
hdr['comment'] = 'This NVO file has been reconstructed from the corresponding ALL file.'
hdr['comment'] = 'All segments have been scaled to match STIS.'
hdr['comment'] = 'The region between 1180 and 1190, affected by the worm, is omitted.'

f.writeto('level0_p117290200000nvo4ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'p117290200000nvo4ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 

filename = 'level0_p117290200000nvo4ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(old['wave'], old['flux'], label='OLD FLUX')
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux']) 
ax2.step(new['wave'], new['flux'])
ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([0,2E-12])